##### Entity Extraction information Validation for 4 entities: Name, Designation, Phone, Address
##### The solution flow to achieve above is pointed below:
    - Extract Signoff/signature text from data
    - Annotation (tagging) for each entity in the signature data
    - Train a blank SpaCy NER model with annotated data 
    - Test and evaluate custom built model



##### Note : Spacy is one of the best NLP packages with accurate and fast performance, and is better than NLTK

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# import joblib
from talon.signature.bruteforce import extract_signature

text='''Subject: california update 2 / 12
executive summary :
the likelihood of there being an involuntary bankruptcy filing against the utilities appears to be greater than 60 % . this is not only due to the circumstances surrounding the filed rate doctrine case mentioned below , but also because the 30 - day cure period during which the utilities have not been paying their bills will end this week , increasing the likelihood that their ipp creditors will act against them . if the state loses the filed rate doctrine case today ( which it is believed will happen ) and there is an involuntary bankruptcy filing ( or even the threat of one ) , this bail - out plan will be enacted quickly .
1 . utilities vs . cpuc
governor davis ' attempt to delay the filed rate doctrine case will not succeed . the case will come before the judge on monday , february 12 th . the federal judge is expected to rule a summary judgment in favor of the utilities . however , the judge will not allow the utilities to collect the injunction release they are requesting ( $ . 01 / kwh ) . this will be left to an appellate court . the decision not to allow the utilities to collect this cash could trigger an involuntary bankruptcy filing , by the smaller ipps ( as noted in our reports last week ) or by larger out - of - state generators such as duke , reliant , and dynegy ( as noted in the press this morning ) . this is expected next week or the week after .
2 . prospects for a bailout
bill ab 18 x is effectively dead from lack of support .

Best,
Jaraslaw Rafalko
Sr. Manager
715 N Tremont Street
Kewanee, IL, 61443
P: (889)-877-5778
'''
body,signature=extract_signature(text)
print("\n\nExtracted Signature : \n\n",signature)



Extracted Signature : 

 Best,
Jaraslaw Rafalko
Sr. Manager
715 N Tremont Street
Kewanee, IL, 61443
P: (889)-877-5778


# Custom NER with Spacy

In [4]:
import pandas as pd
spacy_df=pd.read_excel(r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_try.xlsx','Sheet2')
spacy_df.head()

,Sentence#,Sentence,Word,Tag
0,Sentence: 1,1,Best,Signoff
1,Sentence: 1,1,wishes,Signoff
2,Sentence: 1,1,Jayden,Name
3,Sentence: 1,1,Wilson,Name
4,Sentence: 1,1,Architect,Designation


In [5]:
spacy_df[['Sentence','Word','Tag']].to_csv(r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_try.tsv',sep='\t',index=False,header=0)

# NER Data Creation in Spacy Required Format

In [6]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
input_path=r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_try.tsv'
output_path=r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_try.json'
unknown_label='Other'

f=open(input_path,'r') # input file
fp=open(output_path, 'w') # output file
data_dict={}
annotations =[]
label_dict={}
s=''
start=0
count=1
for line in f:
#     print(line.split('\t')[0],'\n',count,'\n',line.split('\t')[0]==str(count))
    if line.split('\t')[0]==str(count):
#         print('if')
        sent,word,entity=line.split('\t')
#         print(word,entity) # working
        s+=word+" "
        entity=entity[:len(entity)-1]
        if entity!=unknown_label:
            if len(entity) != 1:
                d={}
                d['text']=word
                d['start']=start
                d['end']=start+len(word)-1  
#                 print(d)
                try:
                    label_dict[entity].append(d)
                except:
                    label_dict[entity]=[]
                    label_dict[entity].append(d) 
#                 print(label_dict) #working
        start+=len(word)+1
    else:
#         print('else')
#         print(s)
        data_dict['content']=s
        s=''
        label_list=[]
        for ents in list(label_dict.keys()):
            for i in range(len(label_dict[ents])):
                if(label_dict[ents][i]['text']!=''):
                    l=[ents,label_dict[ents][i]]
                    for j in range(i+1,len(label_dict[ents])): 
                        if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                            di={}
                            di['start']=label_dict[ents][j]['start']
                            di['end']=label_dict[ents][j]['end']
                            di['text']=label_dict[ents][i]['text']
                            l.append(di)
                            label_dict[ents][j]['text']=''
                    label_list.append(l)                          

        for entities in label_list:
            label={}
            label['label']=[entities[0]]
            label['points']=entities[1:]
            annotations.append(label)
        data_dict['annotation']=annotations
        print(data_dict)
        annotations=[]
        json.dump(data_dict, fp)
        fp.write('\n')
        data_dict={}
        start=0
        label_dict={}
        sent,word,entity=line.split('\t')
#         print(word,entity) # working
        s+=word+" "
        entity=entity[:len(entity)-1]
        if entity!=unknown_label:
            if len(entity) != 1:
                d={}
                d['text']=word
                d['start']=start
                d['end']=start+len(word)-1  
#                 print(d)
                try:
                    label_dict[entity].append(d)
                except:
                    label_dict[entity]=[]
                    label_dict[entity].append(d) 
#                 print(label_dict) #working
        start+=len(word)+1
        count=count+1
        


{'content': 'Best wishes Jayden Wilson Architect 42 Fairhaven Commons Way Fairhaven MA 2719 224-828-8773 ', 'annotation': [{'label': ['Signoff'], 'points': [{'text': 'Best', 'start': 0, 'end': 3}]}, {'label': ['Signoff'], 'points': [{'text': 'wishes', 'start': 5, 'end': 10}]}, {'label': ['Name'], 'points': [{'text': 'Jayden', 'start': 12, 'end': 17}]}, {'label': ['Name'], 'points': [{'text': 'Wilson', 'start': 19, 'end': 24}]}, {'label': ['Designation'], 'points': [{'text': 'Architect', 'start': 26, 'end': 34}]}, {'label': ['Address'], 'points': [{'text': '42', 'start': 36, 'end': 37}]}, {'label': ['Address'], 'points': [{'text': 'Fairhaven', 'start': 39, 'end': 47}, {'start': 61, 'end': 69, 'text': 'Fairhaven'}]}, {'label': ['Address'], 'points': [{'text': 'Commons', 'start': 49, 'end': 55}]}, {'label': ['Address'], 'points': [{'text': 'Way', 'start': 57, 'end': 59}]}, {'label': ['Address'], 'points': [{'text': 'MA', 'start': 71, 'end': 72}]}, {'label': ['Address'], 'points': [{'text'

# Email Signature Extraction Example

# Custom NER Model Training

### LABELS :  Name, Designation, Address, Phone, Signoff

In [12]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

input_file=r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_try.json'
training_data = []
lines=[]
with open(input_file, 'r') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    print(text)
    entities = []
    for annotation in data['annotation']:
        point = annotation['points'][0]
        labels = annotation['label']
        if not isinstance(labels, list):
            labels = [labels]

        for label in labels:
            entities.append((point['start'], point['end'] + 1 ,label))
    training_data.append((text, {"entities" : entities}))
print(training_data)

with open(r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_trainingset.pkl', 'wb') as fp:
    pickle.dump(training_data, fp)

Best wishes Jayden Wilson Architect 42 Fairhaven Commons Way Fairhaven MA 2719 224-828-8773 
All the best William Smith Project Lead 30 Memorial Drive Avon MA 2322 201-463-4548 
Best James Johnson Project Manager 250 Hartford Avenue Bellingham MA 2019 205-655-2571 
Best regards Michael Miller Program Manager 137 Teaticket Hwy East Falmouth MA 2536 (218)-839-9958 
Best wishes Jayden Wilson Architect 42 Fairhaven Commons Way Fairhaven MA 2719 224-828-8773 
Fond regards Luis Jackson Technical Specialist 295 Plymouth Street Halifax MA 2338 (254)-405-6210 
Kind regards Jose White Deliver Manager 1775 Washington St Hanover MA 2339 256-416-8947 
Looking forward to hearing from you Ava Martinez Delivery Head 780 Lynnway Lynn MA 1905 Ph - 315-842-9333 
Regards Ashley Robinson Business Analyst 70 Pleasant Valley Street Methuen MA 1844 Ph - (321)-436-8114 
Sincerely Logan Walker Director Technology 72 Main St North Reading MA 1864 Ph - 360-602-3481 
Sincerely yours Aiden Hall Director 200 Otis St

In [2]:

# Training additional entity types using spaCy
from __future__ import unicode_literals, print_function
import pickle
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example


# New entity labels
LABEL=['Address', 'Designation', 'Name', 'Phone', 'Signoff']

# Loading training data 
with open (r'C:\Users\jaysriva\Documents\NER_Spacy_dataset_trainingset.pkl', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

model=None
new_model_name='new_model'
output_dir=r'C:\Users\jaysriva\Documents\'
n_iter=20


"""Setting up the pipeline and entity recognizer, and training the new entity."""


if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
    
if 'ner' not in nlp.pipe_names:
    nlp.add_pipe('ner')
ner = nlp.get_pipe('ner')

for i in LABEL:
    ner.add_label(i)   # Add new entity labels to entity recognizer

if model is None:
    optimizer = nlp.begin_training()
else:
    optimizer = nlp.entity.create_optimizer()

# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3)
        print('Losses', losses)

# # Test the trained model
# test_text = 'Regards Jessica Roberts Analyst 123 Milky Drive San Fransisco CA 76547 (999)-999-999'
# doc = nlp(test_text)
# print("Entities in '%s'" % test_text)
# for ent in doc.ents:
#     print(ent.label_, ent.text)

# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

#     # Test the saved model
#     print("Loading from", output_dir)
#     nlp2 = spacy.load(output_dir)
#     doc2 = nlp2(test_text)
#     for ent in doc2.ents:
#         print(ent.label_, ent.text)
# Our model should not just memorize the training examples. It should learn from them and generalize it to new examples. 
# Once you find the performance of the model satisfactory we save it

Created blank 'en' model
Losses {'ner': 725.1886906167753}
Losses {'ner': 347.46806239123237}
Losses {'ner': 156.50394061728585}
Losses {'ner': 64.870920260854}
Losses {'ner': 59.55504008550626}
Losses {'ner': 37.12619387870269}
Losses {'ner': 27.36496005905821}
Losses {'ner': 22.672198224039498}
Losses {'ner': 19.11560956814476}
Losses {'ner': 15.204584069256546}
Losses {'ner': 31.058734045079003}
Losses {'ner': 31.806199227192327}
Losses {'ner': 7.1103645420424835}
Losses {'ner': 12.7204288524921}
Losses {'ner': 15.105548332365872}
Losses {'ner': 14.830964151819234}
Losses {'ner': 15.880672379499073}
Losses {'ner': 16.610534747465888}
Losses {'ner': 7.236864835943767}
Losses {'ner': 6.061121987319471}
Saved model to C:\Users\jaysriva\Documents\


In [14]:
# Testing

output_dir=Path(output_dir)
nlp2 = spacy.load(output_dir)
lst=['Regards Jessica Roberts Analyst 123 Milky Drive San Fransisco CA 76547 (999)-999-999',
    'Respectfully Jason Mark Senior Director 987 Broad Way NY 11111 P: (987)-788-357',
     'Best Jason Hunter',
     'Regards Kevin',
     'Kendra'
    ]
for i in lst:
    print('Signature : ', i)
    print('\nEntities  - ')
    doc2 = nlp2(i)
    for ent in doc2.ents:
        print(ent.label_, ent.text)
    print('\n\n')

Signature :  Regards Jessica Roberts Analyst 123 Milky Drive San Fransisco CA 76547 (999)-999-999

Entities  - 
Signoff Regards
Name Jessica
Name Roberts
Designation Analyst
Address 123
Address Milky
Address Drive
Address San
Address Fransisco
Address CA
Address 76547
Phone (999)-999-999



Signature :  Respectfully Jason Mark Senior Director 987 Broad Way NY 11111 P: (987)-788-357

Entities  - 
Signoff Respectfully
Name Jason
Name Mark
Designation Senior
Designation Director
Address 987
Address Broad
Address Way
Address NY
Address 11111
Phone P: (987)-788-357



Signature :  Best Jason Hunter

Entities  - 
Signoff Best
Name Jason
Name Hunter



Signature :  Regards Kevin

Entities  - 
Signoff Regards
Name Kevin



Signature :  Kendra

Entities  - 
Name Kendra





In [15]:
!pip install ipython

In [16]:
# Testing
import pandas as pd
from IPython.display import display
from pathlib import Path
import spacy
output_dir=r'C:\Users\jaysriva\Documents\"
output_dir=Path(output_dir)
nlp2 = spacy.load(output_dir)
lst=['Best William Smith Project Lead 30 Memorial Drive Avon MA 2322 201-463-4548 ',
     'Respectfully Jason Mark Senior Director 987 Broad Way NY 11111 P: (987)-788-357',
     'Best Jason',
     'Regards Kevin',
     'Kendra'
    ]
for i in lst:
    NER_data=pd.DataFrame()
    print('Signature : ', i)
    print('\nEntities  - ')
    doc2 = nlp2(i)
    NER_data['Words'] = [ent.text for ent in doc2.ents]
    NER_data['Labels'] = [ent.label_ for ent in doc2.ents]
    display(NER_data)\
    

Signature :  Best William Smith Project Lead 30 Memorial Drive Avon MA 2322 201-463-4548 

Entities  - 


,Words,Labels
0,Best,Signoff
1,William,Name
2,Smith,Name
3,Project,Designation
4,Lead,Designation
5,30,Address
6,Memorial,Address
7,Drive,Address
8,Avon,Address
9,MA,Address


Signature :  Respectfully Jason Mark Senior Director 987 Broad Way NY 11111 P: (987)-788-357

Entities  - 


,Words,Labels
0,Respectfully,Signoff
1,Jason,Name
2,Mark,Name
3,Senior,Designation
4,Director,Designation
5,987,Address
6,Broad,Address
7,Way,Address
8,NY,Address
9,11111,Address


Signature :  Best Jason

Entities  - 


,Words,Labels
0,Best,Signoff
1,Jason,Name


Signature :  Regards Kevin

Entities  - 


,Words,Labels
0,Regards,Signoff
1,Kevin,Name


Signature :  Kendra

Entities  - 


,Words,Labels
0,Kendra,Name


# Model Evaluation

In [17]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        print(pred,annot)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores


# example run

examples = [
    ('Best wishes Jayson Dikson President 39 Fairhaven Commons Way Fairhaven CA 3126 987-828-8773 ', {'entities': [(0, 4, 'Signoff'), (5, 11, 'Signoff'), (12, 18, 'Name'), (19, 25, 'Name'), (26, 35, 'Designation'), (36, 38, 'Address'), (39, 48, 'Address'), (49, 56, 'Address'), (57, 60, 'Address'), (71, 73, 'Address'), (74, 78, 'Address'), (79, 91, 'Phone')]}),
    ('Looking forward Andorson 1151 S Alabama Ave Monroeville WA 76767 ', {'entities': [(0, 7, 'Signoff'), (8, 15, 'Signoff'), (16, 24, 'Name'), (25, 29, 'Address'), (30, 31, 'Address'), (32, 39, 'Address'), (40, 43, 'Address'), (44, 55, 'Address'), (56, 58, 'Address'), (59, 64, 'Address')]})
]

ner_model = spacy.load(r'C:\Users\jaysriva\Documents') # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, examples)
print(results)

Best wishes Jayson Dikson President 39 Fairhaven Commons Way Fairhaven CA 3126 987-828-8773  {'entities': [(0, 4, 'Signoff'), (5, 11, 'Signoff'), (12, 18, 'Name'), (19, 25, 'Name'), (26, 35, 'Designation'), (36, 38, 'Address'), (39, 48, 'Address'), (49, 56, 'Address'), (57, 60, 'Address'), (71, 73, 'Address'), (74, 78, 'Address'), (79, 91, 'Phone')]}
Looking forward Andorson 1151 S Alabama Ave Monroeville WA 76767  {'entities': [(0, 7, 'Signoff'), (8, 15, 'Signoff'), (16, 24, 'Name'), (25, 29, 'Address'), (30, 31, 'Address'), (32, 39, 'Address'), (40, 43, 'Address'), (44, 55, 'Address'), (56, 58, 'Address'), (59, 64, 'Address')]}
{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_per_feat': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None, 'cats_score': 0.0, 'cats_score_desc'

# Model Load and Entity Recognition Function

In [18]:
from talon.signature.bruteforce import extract_signature
import joblib
from spacy import displacy
from pathlib import Path
import time
import spacy
import warnings
warnings.filterwarnings("ignore")

def Custom_NER_Prediction(text):
#     print('Loading Custom NER Model...')
    time.sleep(1)
    output_dir=r'C:\Users\jaysriva\Documents'
    output_dir=Path(output_dir)
    nlp2 = spacy.load(output_dir)

    CStart_text='\033[34m'
    CStart_heading='\033[1m\033[37m' #1m bold, 37,97 grey text, 47,107 grey bg, 44 blue bg
    CEnd='\033[0m'

#     print(CStart_heading + '\nText : ' + CEnd + '\n' + CStart_text + text + CEnd)
    text, signature = extract_signature(text)
    if signature!= None:
        signature=signature.replace('\n',' ')
        signature=signature.replace(',','')
        signature=signature.replace('.','')


    #     print(CStart_heading +'\nExtracted Signature from Text : \n' + CEnd +  CStart_text +  signature + CEnd, '\n')

        doc2 = nlp2(signature)
    #     print(CStart_heading+'\nExtracted Entities from Signature : \n' + CEnd)
        # displacy.serve(doc2, style="ent")
        colors = {"Phone": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
                 "Name": "linear-gradient(90deg, #6190E8, #A7BFE8)",
                 "Designation": "linear-gradient(90deg,#e2ebf0,#FF7F50)",
                 "Address": "linear-gradient(90deg,#cfd9df,#e2ebf0)",#" )",
                  "Signoff": "linear-gradient(180deg, #43C6AC, #DBE6F6)"
                 }
        options = {"ents": ['Address', 'Designation', 'Name', 'Phone', 'Signoff'], "colors": colors}
        displacy.render(doc2, style="ent", options=options) # Jupyter is already running on server so we use render
        # displacy.serve(doc2, style="ent", options=options) # To create server
        # for ent in doc2.ents:
        #     print("('" + '\033[34m' + ent.text.ljust(20) + '\033[0m' + "', '" + '\033[77m' + ent.label_ + '\033[0m'+ "'')")
    else:
        print('No Signature Found!')

# *************************  Predictions  ************************* 

In [19]:
text='''Subject: california update 2 / 12
executive summary :
the likelihood of there being an involuntary bankruptcy filing against the utilities appears to be greater than 60 % . this is not only due to the circumstances surrounding the filed rate doctrine case mentioned below , but also because the 30 - day cure period during which the utilities have not been paying their bills will end this week , increasing the likelihood that their ipp creditors will act against them . if the state loses the filed rate doctrine case today ( which it is believed will happen ) and there is an involuntary bankruptcy filing ( or even the threat of one ) , this bail - out plan will be enacted quickly .
1 . utilities vs . cpuc
governor davis ' attempt to delay the filed rate doctrine case will not succeed . the case will come before the judge on monday , february 12 th . the federal judge is expected to rule a summary judgment in favor of the utilities . however , the judge will not allow the utilities to collect the injunction release they are requesting ( $ . 01 / kwh ) . this will be left to an appellate court . the decision not to allow the utilities to collect this cash could trigger an involuntary bankruptcy filing , by the smaller ipps ( as noted in our reports last week ) or by larger out - of - state generators such as duke , reliant , and dynegy ( as noted in the press this morning ) . this is expected next week or the week after .
2 . prospects for a bailout
bill ab 18 x is effectively dead from lack of support .

Best,
William Smith
Project Lead 
30 Memorial Drive Avon MA 2322
P: (01)-201-463-4548 
'''
# Entity Recognition using Custon trained Spacy NER Model
Custom_NER_Prediction(text)

### Signature Type 1: Entities Available - Name Designation Address Phone

In [20]:
text='''Subject: california update 2 / 12
executive summary :
the likelihood of there being an involuntary bankruptcy filing against the utilities appears to be greater than 60 % . this is not only due to the circumstances surrounding the filed rate doctrine case mentioned below , but also because the 30 - day cure period during which the utilities have not been paying their bills will end this week , increasing the likelihood that their ipp creditors will act against them . if the state loses the filed rate doctrine case today ( which it is believed will happen ) and there is an involuntary bankruptcy filing ( or even the threat of one ) , this bail - out plan will be enacted quickly .
1 . utilities vs . cpuc
governor davis ' attempt to delay the filed rate doctrine case will not succeed . the case will come before the judge on monday , february 12 th . the federal judge is expected to rule a summary judgment in favor of the utilities . however , the judge will not allow the utilities to collect the injunction release they are requesting ( $ . 01 / kwh ) . this will be left to an appellate court . the decision not to allow the utilities to collect this cash could trigger an involuntary bankruptcy filing , by the smaller ipps ( as noted in our reports last week ) or by larger out - of - state generators such as duke , reliant , and dynegy ( as noted in the press this morning ) . this is expected next week or the week after .
2 . prospects for a bailout
bill ab 18 x is effectively dead from lack of support .

Best,
Jaraslaw Rafalko
Sr. Manager
715 N Tremont Street
Kewanee, IL, 61443
P: (889)-877-5778
'''

# Calling Custom NER Prediction Function
Custom_NER_Prediction(text)

In [21]:
text='''Subject: key hr issues going forward
a ) year end reviews - report needs generating like mid - year documenting business unit performance on review completion - david to john ;
b ) work out or plan generation for the nim / issues employees - david to john ;
c ) hpl transition issues - ongoing .
officially transferred .
Thanks,
Delainey Smith
Associate Director
11495 Wills Creek Road, San Diego, CA. 92131
P: (789) 698-9876'''
Custom_NER_Prediction(text)

### Signature With Name Designation Phone

In [22]:
text='''Subject: congrats !
contratulations on the execution of the central maine sos deal ! this is another great example of what we can do when everyone comes together to get something done . this transaction brings both strategic value to the business , nice positions for the book and quite a nice chunk of change as well !
great job guys !
( hey dana , are you paying for the celebration dinner ? ! )

Thanks,
Kristin Spivey
Sr. Director
P: (989) 878-9898'''
Custom_NER_Prediction(text)

### Signature with Designation Address Phone

In [23]:
text='''Gentle Reminder! 

Best wishes,

Associate Analyst
98765 Fifth Avenue, Manhattan, NY. 100110
(989) 878-9898'''
Custom_NER_Prediction(text)

### Signature With only Name

In [24]:
text='''Gentle Reminder! 

Regards,
John Miller
'''
Custom_NER_Prediction(text)

### Signature With only First Name

In [25]:
text='''Good Job!

Thanks,
Kevin'''
Custom_NER_Prediction(text)

### Signature Type 5: With First Letter of Name

In [26]:
text='''Gentle Reminder! 

Regards,
J
'''
Custom_NER_Prediction(text)

### Signature Type 6: With Only Phone Number

In [27]:
text='''Gentle Reminder! 

Best wishes,

(989) 878-9898'''
Custom_NER_Prediction(text)

### Signature With Only (Address & Phone Number)

In [28]:
text='''Gentle Reminder! 

Best wishes,

98765 Fifth Avenue, Manhattan, NY. 100110
(989) 878-9898'''
Custom_NER_Prediction(text)

### Signature Type 9: With No Entites

In [29]:
text='''Gentle Reminder! 
Best wishes,

'''
Custom_NER_Prediction(text)

### Type 10: With No Signature

In [30]:
text='''Gentle Reminder! 

'''
Custom_NER_Prediction(text)

No Signature Found!


### Signature With Changed Order of Entites (Name Designation Phone Address)

In [31]:
text='''Gentle Reminder! 

Regards,
Delainey Smith
Associate Director
P: (789) 698-9876
11495 Wills Creek Road, San Diego, CA. 92131

'''
Custom_NER_Prediction(text)

### Signature With Changed Order of Entites (Name Address Phone Designation)

In [32]:
text='''Gentle Reminder! 

Regards,
Delainey Smith
11495 Wills Creek Road, San Diego, CA. 92131
P: (789) 698-9876
Associate Director
'''
Custom_NER_Prediction(text)